This is the Python implementation of the 3D-RFT Code based on the framework
proposed by Agarwal et al. --> https://doi.org/10.1073/pnas.2214017120

In [158]:
# Main file for executing the analysis of the RFT data
######## IMPORTS ########
import math

import ipywidgets as widgets
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly as py
import plotly.graph_objects as go
import plotly.figure_factory as ff
import scipy as sp
import seaborn as sns
from IPython.display import display
from mpl_toolkits import mplot3d


In [159]:
from src import rft_functions
from src import helper_functions


In [160]:
######## INPUTS ########

## Model selection
MODEL = "CylinderRough"

## Physical properties
BULK_DENSITY = 1310
FRICTION_MATERIAL = 0.21
FRICTION_SURFACE = 0.4
FRICTION_TYPE = "coefficient"
if FRICTION_TYPE == "angle":
    FRICTION_MATERIAL = math.tan(np.deg2rad(FRICTION_MATERIAL))
    FRICTION_SURFACE = math.tan(np.deg2rad(FRICTION_SURFACE))
elif FRICTION_TYPE == "coefficient":
    pass
else:
    raise ValueError("FRICTION_UNIT must be either 'angle' or 'coefficient'")
GRAVITY = 9.81
MATERIAL_CONSTANT = (
    GRAVITY
    * BULK_DENSITY
    * (
        894 * FRICTION_MATERIAL**3
        - 386 * FRICTION_MATERIAL**2
        + 89 * FRICTION_MATERIAL
    )
) * 10**-9

## Movement defintions
LINEAR_VELOCITY = 0.1
DIR_ANGLE_XZ_DEG = -90
DIR_ANGLE_Y_DEG = -90

ROTATION = True
ANGULAR_VELOCITY = np.array([0, 0, -2 * sp.pi])

## Depth parameters in mm
DEPTH = 100

## Miscellaneous & Plots
THRESHOLD = 1.0e-12


In [161]:
%%html
<style>
.cell-output-ipywidget-background {
   background-color: transparent !important;
}
.jp-OutputArea-output {
   background-color: transparent;
}
</style>


In [162]:
# STL Processing
(
    point_list,
    normal_list,
    area_list,
    depth_list,
    object_width_x,
    object_width_y,
    object_height,
    vertices,
    faces,
    trg,
) = rft_functions.import_mesh(MODEL)
point_list[:, 2] -= DEPTH
vertices[:, 2] -= DEPTH


In [163]:
# Creating a 3D plot using Plotly
fig = go.FigureWidget()

fig.add_trace(
    go.Mesh3d(
        x=vertices[:, 0],
        y=vertices[:, 1],
        z=vertices[:, 2],
        i=faces[:, 0],
        j=faces[:, 1],
        k=faces[:, 2],
        colorscale="Plasma",
        intensity=vertices[:, 2],
        cmin=np.min(vertices[:, 2]),
        cmax=np.max(vertices[:, 2]),
    )
)

# Updating the layout of the plot
fig.update_layout(
    template="plotly_dark",
    width=600,
    height=500,
    scene=dict(
        aspectmode="data",
        aspectratio=dict(
            x=np.ptp(vertices[:, 0]),
            y=np.ptp(vertices[:, 1]),
            z=np.ptp(vertices[:, 2]),
        ),
    ),
)

display(fig)


FigureWidget({
    'data': [{'cmax': -50.01523,
              'cmin': -99.98477,
              'colorscale': [[0.0, '#0d0887'], [0.1111111111111111, '#46039f'],
                             [0.2222222222222222, '#7201a8'], [0.3333333333333333,
                             '#9c179e'], [0.4444444444444444, '#bd3786'],
                             [0.5555555555555556, '#d8576b'], [0.6666666666666666,
                             '#ed7953'], [0.7777777777777778, '#fb9f3a'],
                             [0.8888888888888888, '#fdca26'], [1.0, '#f0f921']],
              'i': array([ 42,   0,   0, ..., 672, 672, 672], dtype=int64),
              'intensity': array([-75.     , -75.     , -75.     , ..., -73.25609, -76.74391, -73.25609],
                                 dtype=float32),
              'j': array([380, 380, 381, ..., 780, 779, 484], dtype=int64),
              'k': array([  0, 381,  43, ..., 779, 778, 673], dtype=int64),
              'type': 'mesh3d',
              'uid': '98222d6

In [164]:
## Calculate movement
movement = rft_functions.calc_movement(
    point_list,
    DEPTH,
    object_height,
    DIR_ANGLE_XZ_DEG,
    DIR_ANGLE_Y_DEG,
    LINEAR_VELOCITY,
    ROTATION,
    ANGULAR_VELOCITY,
)


In [165]:
## Check conditions
(
    point_list,
    normal_list,
    area_list,
    depth_list,
    movement,
) = rft_functions.check_conditions(
    point_list, normal_list, area_list, depth_list, movement
)


In [166]:
fig = go.FigureWidget()
fig.add_trace(
    go.Cone(
        x=point_list[:, 0],
        y=point_list[:, 1],
        z=point_list[:, 2],
        u=movement[:, 0],
        v=movement[:, 1],
        w=movement[:, 2],
        sizeref=3,
        showscale=False,
        colorscale="Teal",
    )
)

# Updating the layout of the plot
fig.update_layout(
    template="plotly_dark",
    width=600,
    height=500,
    scene=dict(
        aspectmode="data",
        aspectratio=dict(
            x=np.ptp(vertices[:, 0]),
            y=np.ptp(vertices[:, 1]),
            z=np.ptp(vertices[:, 2]),
        ),
    ),
)

display(fig)


FigureWidget({
    'data': [{'colorscale': [[0.0, 'rgb(209, 238, 234)'], [0.16666666666666666,
                             'rgb(168, 219, 217)'], [0.3333333333333333, 'rgb(133,
                             196, 201)'], [0.5, 'rgb(104, 171, 184)'],
                             [0.6666666666666666, 'rgb(79, 144, 166)'],
                             [0.8333333333333334, 'rgb(59, 115, 143)'], [1.0,
                             'rgb(42, 86, 116)']],
              'showscale': False,
              'sizeref': 3,
              'type': 'cone',
              'u': array([-0.9750784 , -0.97416377, -0.97324437, ...,  0.96403446,  0.96403446,
                           0.95514356]),
              'uid': '7d28ced7-469b-48c2-a779-bed942d3ae33',
              'v': array([-0.00849426, -0.04412649, -0.06188812, ...,  0.15031517,  0.15031517,
                           0.20134536]),
              'w': array([-0.22169792, -0.22148997, -0.22128093, ..., -0.21918693, -0.21918693,
                          -

In [167]:
## Find local coordinate frame for each subsurface
z_local, r_local, theta_local = rft_functions.find_local_frame(normal_list, movement)


In [168]:
## Find the characteristic angles of the RFT method
beta, gamma, psi = rft_functions.find_angles(
    normal_list, movement, z_local, r_local, theta_local
)


In [169]:
## Find empirical values for the RFT method
f_1, f_2, f_3 = rft_functions.find_fit(beta, gamma, psi)


In [170]:
## Find dimensionless response vectors alpha
alpha_generic, alpha_generic_n, alpha_generic_t, alpha = rft_functions.find_alpha(
    normal_list,
    movement,
    beta,
    gamma,
    psi,
    z_local,
    r_local,
    theta_local,
    f_1,
    f_2,
    f_3,
    MATERIAL_CONSTANT,
    FRICTION_SURFACE,
)


In [171]:
forces, pressure, force_x, force_y, force_z, resultant = rft_functions.find_forces(
    alpha, depth_list, area_list
)


In [172]:
torques, torque_x, torque_y, torque_z, resultant_torque = rft_functions.find_torques(
    point_list, forces
)
